In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
directories = {
    "SSC": "../ssc",
    "DVS Gesture": "../dvs_gesture",
    "HSD": "../hsd",
}

collected_frames = dict()
for dataset, path in directories.items():
    df = pd.read_csv(Path(path) / "times.csv", header=0, index_col=0)
    collected_frames[dataset] = pd.concat(
        [
            pd.DataFrame({"algorithm": c, "time": df[c], "Dataset": dataset})
            for c in df.columns
        ],
        ignore_index=True
    )

times = pd.concat(collected_frames.values(), ignore_index=True)

In [ ]:
# sns.set(font_scale=1.5, style="white")
plot = sns.barplot(data=times, x="time", y="Dataset", hue="algorithm")
plot.set_xlabel("Time per epoch (s)")
plot.axes.yaxis.label.set_visible(False)
plot.axes.set_yticklabels(plot.axes.get_yticklabels(), rotation=45, va="center")
plt.tight_layout()
plt.savefig("comp_time_comparison.svg")

In [ ]:
for dataset, df in collected_frames.items():
    print(dataset)
    grouped = df.groupby(["algorithm"])
    means = grouped.mean() # .reset_index()
    stds = grouped.std()
    fig = plt.figure(figsize=(2.5, 3))
    x = ['BPTT', 'SLAYER', 'EXODUS']
    y = [means.loc[method]["time"] for method in x]
    s = [stds.loc[method]["time"] for method in x]
    b = np.array(y[1:])
    db = np.array(s[1:])
    ratios = y[0] / b
    s_ratios = np.sqrt(s[0]**2 / b**2 + y[0]**2 * db**2 / b**4)
    print(f"\tBPTT: {y[0]:.2f} +- {s[0]:.2f}")
    print(f"\tSLAYER: {y[1]:.2f} +- {s[1]:.2f}")
    print(f"\t\t {ratios[0]:.2f} +- {s_ratios[0]:.2f} times faster")
    print(f"\tEXODUS: {y[2]:.2f} +- {s[2]:.2f}")
    print(f"\t\t {ratios[1]:.2f} +- {s_ratios[1]:.2f} times faster")
    barlist=plt.bar(x, y, yerr=s) #, width=1)
    barlist[0].set_color('C2')
    barlist[1].set_color('C1')
    barlist[2].set_color('C0')
    ax = fig.get_axes()[0]
    ax.set_xticklabels(['BPTT', 'SLAYER', 'EXODUS'], rotation=20)
    ax.set_ylabel("Time / epoch (s)")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    plt.savefig(f"time_{dataset}.svg")
    
    

In [ ]:
df = collected_frames["SSC"]
grouped = df.groupby(["algorithm"])
means = grouped.mean() # .reset_index()
stds = grouped.std().reset_index()
x = ['BPTT', 'SLAYER', 'EXODUS']
y = [means.loc[method]["time"] for method in x]

In [ ]:
y